In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import shapiro

In [ ]:
'''
# --- 정규성 검정 ---
# - Group별 age 데이터
age1 = df.loc[df['survived'] == 1, 'age'].dropna()
age0 = df.loc[df['survived'] == 0, 'age'].dropna()

# - Shapiro 검정
stat1, p1 = shapiro(age1)
stat0, p0 = shapiro(age0)

# - 소수점 둘째 자리까지 반올림하여 출력
print(f"Survived=1: statistic={stat1:.2f}, pvalue={p1:.2f}")
print(f"Survived=0: statistic={stat0:.2f}, pvalue={p0:.2f}")

# - 전체 데이터 형태 확인
print("DataFrame shape:", df.shape)

# --- 등분산성 검정 ---
from scipy.stats import levene

# - 그룹별 age 데이터
group1 = df.loc[df['survived'] == 1, 'age'].dropna()
group2 = df.loc[df['survived'] == 0, 'age'].dropna()

# - Levene 검정
res = levene(group1, group2)

# - 통계량과 p-value 추출 후 소수점 둘째 자리까지 반올림해 출력
stat, pvalue = res.statistic, res.pvalue
print(f"Levene test: statistic={stat:.2f}, pvalue={pvalue:.2f}")

'''